In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

import os
from hampel import hampel
import sys
sys.path.append('../')

from dataloader.dataloader import *
from training.training import *
from utils.utils import *
from visualizations.visualizations import *
from evaluation.evaluation import *

In [2]:
data_folder = "../Data/Data v5"

df = pd.read_csv(os.path.join(data_folder, "amari_ue_data_final_v5.csv"))
df = df.sort_values(["imeisv", "_time"], ascending = True)

/tmp/ipykernel_177994/1795721224.py:3: DtypeWarning: Columns (16,24,26,27,32,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_folder, "amari_ue_data_final_v5.csv"))


In [3]:
df['imeisv'] = df['imeisv'].astype(str)

In [4]:
feature_columns = [
    'dl_bitrate','ul_bitrate', 
    'cell_x_dl_retx', 'cell_x_dl_tx',
    'cell_x_ul_retx', 'cell_x_ul_tx',
    'ul_total_bytes_non_incr', 'dl_total_bytes_non_incr'
    ]

store_columns = ['_time', 'imeisv'] + feature_columns

### Apply Smoothing

In [5]:
imeisv_df_for_ma = {}

for imeisv, imeisv_df in df.groupby('imeisv'):
    imeisv_df[feature_columns] = imeisv_df[feature_columns].rolling(window=10).mean()

    imeisv_df_for_ma[str(imeisv)] = imeisv_df
            
imeisv_ma_df = pd.concat(list(imeisv_df_for_ma.values()))

In [6]:
imeisv_ma_df.to_csv(os.path.join(data_folder, "amari_ue_data_final_v5_smoothed.csv"), index = False)

### Scaling TS

In [7]:
imeisv_dfs_scaled = {}

for imeisv, imeisv_df in df.groupby('imeisv'):
    scaler = StandardScaler()
    
    scaled_data = scaler.fit_transform(imeisv_df[feature_columns])
    imeisv_df[feature_columns] = scaled_data
        
    imeisv_dfs_scaled[imeisv] = imeisv_df

3557821101183501
8609960468879057
8609960480666910
8609960480859057
8628490433231157
8642840401594200
8642840401612300
8642840401624200
8677660403123800


In [8]:
final_df = pd.concat(list(imeisv_dfs_scaled.values()))

In [9]:
final_df.to_csv(os.path.join(data_folder, "amari_ue_data_final_v5_smoothed_scaled.csv"), index = False)

### Scaling TS (separately)

In [ ]:
imeisv_df_no_outliers_scaled_sep = {}

for imeisv, df in imeisv_df_for_ma.items():
    benign_df = df[df['label'] == 0].copy()
    malicious_df = df[df['label'] == 1].copy()
    
    benign_scaler = StandardScaler()
    malicious_scaler = StandardScaler()
    
    scaled_benign_data = benign_scaler.fit_transform(benign_df[feature_columns])
    benign_df[feature_columns] = scaled_benign_data
    
    if malicious_df.shape[0] > 0:
        scaled_malicious_data = malicious_scaler.fit_transform(malicious_df[feature_columns])
        malicious_df[feature_columns] = scaled_malicious_data
    
    imeisv_df_no_outliers_scaled_sep[imeisv] = pd.concat(
        [benign_df,malicious_df], axis = 0
        ).sort_values(['imeisv','_time'], ascending= True)

In [ ]:
final_df = pd.concat(list(imeisv_df_no_outliers_scaled_sep.values()))

In [ ]:
final_df.to_csv(os.path.join(data_folder, "amari_ue_data_final_v5_smoothed_scaled_sep.csv"), index = False)